In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score, make_scorer
from sklearn.model_selection import cross_validate, train_test_split

# Gaussian Process Regression for Time Series Analysis

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels \
    import RBF, WhiteKernel, RationalQuadratic, ExpSineSquared

## Load Dataset

This example is based on Section 5.4.3 of “Gaussian Processes for Machine Learning”. It illustrates an example of complex kernel engineering and hyperparameter optimization using gradient ascent on the log-marginal-likelihood. The data consists of the monthly average atmospheric CO2 concentrations (in parts per million by volume (ppmv)) collected at the Mauna Loa Observatory in Hawaii, between 1958 and 2001. The objective is to model the CO2 concentration as a function of the time t.

In [ ]:
def load_mauna_loa_atmospheric_co2():
    ml_data = fetch_openml(data_id=41187)
    months = []
    ppmv_sums = []
    counts = []

    y = ml_data.data[:, 0]
    m = ml_data.data[:, 1]
    month_float = y + (m - 1) / 12
    ppmvs = ml_data.target

    for month, ppmv in zip(month_float, ppmvs):
        if not months or month != months[-1]:
            months.append(month)
            ppmv_sums.append(ppmv)
            counts.append(1)
        else:
            # aggregate monthly sum to produce average
            ppmv_sums[-1] += ppmv
            counts[-1] += 1

    months = np.asarray(months).reshape(-1, 1)
    avg_ppmvs = np.asarray(ppmv_sums) / counts
    return months, avg_ppmvs

In [ ]:
X, y = load_mauna_loa_atmospheric_co2()
print(X.shape, y.shape)

## Split Dataset

Here, we train on the first 80% of the time series and predict the future 20% (temporal split).

In [ ]:
n_examples = y.shape[0]
train_idx = int(0.8 * n_examples)

X_train = X[:train_idx]
y_train = y[:train_idx]
X_test = X[train_idx:]
y_test = y[train_idx:]
print(X_train.shape, )

## Kernel Construction

The kernel is composed of several terms that are responsible for explaining different properties of the signal:

- a long term, smooth rising trend is to be explained by an RBF kernel. The RBF kernel with a large length-scale enforces this component to be smooth; it is not enforced that the trend is rising which leaves this choice to the GP. The specific length-scale and the amplitude are free hyperparameters.

- a seasonal component, which is to be explained by the periodic ExpSineSquared kernel with a fixed periodicity of 1 year. The length-scale of this periodic component, controlling its smoothness, is a free parameter. In order to allow decaying away from exact periodicity, the product with an RBF kernel is taken. The length-scale of this RBF component controls the decay time and is a further free parameter.

- smaller, medium term irregularities are to be explained by a RationalQuadratic kernel component, whose length-scale and alpha parameter, which determines the diffuseness of the length-scales, are to be determined. According to [RW2006], these irregularities can better be explained by a RationalQuadratic than an RBF kernel component, probably because it can accommodate several length-scales.

- a “noise” term, consisting of an RBF kernel contribution, which shall explain the correlated noise components such as local weather phenomena, and a WhiteKernel contribution for the white noise. The relative amplitudes and the RBF’s length scale are further free parameters.


In [ ]:
k1 = 50.0**2 * RBF(length_scale=50.0)  # long term smooth rising trend
k2 = 2.0**2 * RBF(length_scale=100.0) \
    * ExpSineSquared(length_scale=1.0, periodicity=1.0,
                     periodicity_bounds="fixed")  # seasonal component
# medium term irregularities
k3 = 0.5**2 * RationalQuadratic(length_scale=1.0, alpha=1.0)
k4 = 0.1**2 * RBF(length_scale=0.1) \
    + WhiteKernel(noise_level=0.1**2,
                  noise_level_bounds=(1e-3, np.inf))  # noise terms
kernel = k1 + k2 + k3 + k4

# Train model

We optimize the kernel hyperparameters by maximizing the model log likelihood.

In [ ]:
gp = GaussianProcessRegressor(kernel=kernel, alpha=0,
                              normalize_y=True)
gp.fit(X_train, y_train)

## Evaluate model

We can evaluate our model by computing the R2 score and mean absolute error on the train and test set. We then plot our model forecast with the true measurements.

In [ ]:
y_train_pred = gp.predict(X_train)
y_test_pred = gp.predict(X_test)
print("train r2_score:", r2_score(y_train, y_train_pred), "train mae: ", mean_absolute_error(y_train, y_train_pred))
print("test r2_score:", r2_score(y_test, y_test_pred), "test mae: ", mean_absolute_error(y_test, y_test_pred))

In [ ]:
X_ = np.linspace(X.min(), X.max(), 1000)[:, np.newaxis]
y_pred, y_std = gp.predict(X_, return_std=True)

# Illustration
plt.plot(X, y, c='r')
plt.plot(X_, y_pred)
plt.fill_between(X_[:, 0], y_pred - y_std, y_pred + y_std,
                 alpha=0.5, color='k')
plt.xlim(X_.min(), X_.max())
plt.xlabel("Year")
plt.ylabel(r"CO$_2$ in ppm")
plt.title(r"Atmospheric CO$_2$ concentration at Mauna Loa")
plt.tight_layout()
plt.show()

## Re-train Model for Prospective Use

We can retrain the model with the entire dataset and then use the retrained model to predict out to 2030. Note how the uncertainty (gray envelope) increases as we project out further in time.

In [ ]:
k1 = 50.0**2 * RBF(length_scale=50.0)  # long term smooth rising trend
k2 = 2.0**2 * RBF(length_scale=100.0) \
    * ExpSineSquared(length_scale=1.0, periodicity=1.0,
                     periodicity_bounds="fixed")  # seasonal component
# medium term irregularities
k3 = 0.5**2 * RationalQuadratic(length_scale=1.0, alpha=1.0)
k4 = 0.1**2 * RBF(length_scale=0.1) \
    + WhiteKernel(noise_level=0.1**2,
                  noise_level_bounds=(1e-3, np.inf))  # noise terms
kernel = k1 + k2 + k3 + k4

In [ ]:
gp = GaussianProcessRegressor(kernel=kernel, alpha=0,
                              normalize_y=True)
gp.fit(X, y)

In [ ]:
print("\nLearned kernel: %s" % gp.kernel_)
print("Log-marginal-likelihood: %.3f"
      % gp.log_marginal_likelihood(gp.kernel_.theta))

In [ ]:
X_ = np.linspace(X.min(), X.max() + 30, 1000)[:, np.newaxis]
y_pred, y_std = gp.predict(X_, return_std=True)

# Illustration
plt.scatter(X, y, c='k')
plt.plot(X_, y_pred)
plt.fill_between(X_[:, 0], y_pred - y_std, y_pred + y_std,
                 alpha=0.5, color='k')
plt.xlim(X_.min(), X_.max())
plt.xlabel("Year")
plt.ylabel(r"CO$_2$ in ppm")
plt.title(r"Atmospheric CO$_2$ concentration at Mauna Loa")
plt.tight_layout()
plt.show()

# Random Forest Regression for Protein-Ligand Binding Energy Prediction

## Software installation

Although Google Colab comes with many useful numerical libraries pre-installed, other libraries require external installation. Here, we install the `rdkit` open source cheminformatics packages for performing feature engineering with the BACE dataset.

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!time bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!time conda install -q -y -c conda-forge rdkit

import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
from scipy import stats
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import RandomForestRegressor

## Load Dataset

Let's load up the BACE dataset.

In [ ]:
os.system("wget https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/desc_canvas_aug30.csv")
df = pd.read_csv("desc_canvas_aug30.csv")
df

In [ ]:
label = 'pIC50'
y = df[label].values
smiles = df['mol'].values

## Feature Engineering

We compute the topological fingerprint of each molecule in the dataset using the Extended Connectivity Fingerprint (ECFP) algorithm.

<img src="https://miro.medium.com/max/1180/1*oaJ6HRYeCImh7TmMrS345Q.png">

In [ ]:
fps = []
for s in smiles:
  m = Chem.MolFromSmiles(s)
  fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=768)
  fps.append(fp)
X = np.stack(fps, axis=0)
print(X.shape, y.shape)

## Dataset Splitting and Transformation

We split the dataset into 80% training and 20% testing. We standardize the pIC50 values using the mean and stdev from the train set.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

y_mean = y_train.mean()
y_std = y_train.std()

y_train_t = (y_train - y_mean) / y_std
y_test_t = (y_test - y_mean) / y_std

## Cross-validation

We perform 5-fold cross-validation using the training set. Here, we see how the model improves by adding more estimators or "trees".

In [ ]:
scoring = {'r2_score': make_scorer(r2_score)}
for est in [10, 20, 40, 80, 160]:
  cv = cross_validate(RandomForestRegressor(n_estimators=est, criterion='mse', max_features=0.33), 
                      X_train, y_train_t, scoring=scoring, return_train_score=True)
  for score in ['train_r2_score', 'test_r2_score']:
    scores = cv[score]
    mean_scores = np.round(np.mean(scores), 4)
    stderr_scores = np.round(stats.sem(scores), 4)
    print("n_estimators: {} {}: {} +/- {}".format(est, score, mean_scores, stderr_scores))

## Model Evaluation

We evaluate the final model using the train and (unseen) test set.

In [ ]:
reg = RandomForestRegressor(n_estimators=160, criterion='mse', max_features=0.33)
reg.fit(X_train, y_train_t)
print(reg.score(X_train, y_train_t))
print(reg.score(X_test, y_test_t))

In [ ]:
y_pred_train = reg.predict(X_train)*y_std + y_mean
y_pred_test = reg.predict(X_test)*y_std + y_mean
print("train r2:", r2_score(y_train, y_pred_train), "train mae:", mean_absolute_error(y_train, y_pred_train))
print("test r2:", r2_score(y_test, y_pred_test), "test mae:", mean_absolute_error(y_test, y_pred_test))

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 7.5))
ax[0].scatter(y_train, y_pred_train)
ax[0].plot(np.linspace(0, 10), np.linspace(0, 10), 'r')
ax[0].set_aspect('equal', adjustable='box')
ax[0].set_xlabel("Log Solubility")
ax[0].set_ylabel("Predicted Log Solubility")
ax[0].set_title("Train R2: {}".format(r2_score(y_train, y_pred_train)))
ax[1].scatter(y_test, y_pred_test)
ax[1].plot(np.linspace(0, 10), np.linspace(0, 10), 'r')
ax[1].set_aspect('equal', adjustable='box')
ax[1].set_xlabel("Log Solubility")
ax[1].set_ylabel("Predicted Log Solubility")
ax[1].set_title("Test R2: {}".format(r2_score(y_test, y_pred_test)))
plt.show()
plt.close()

## Visualizing Feature Importance

We can use the computed feature importances from the `RandomForestRegressor` and visualize the corresponding molecular substructors.

In [ ]:
fi = np.array(reg.feature_importances_)
sortidx = np.argsort(fi)
print(fi[sortidx[-1]])

In [ ]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
bi = {}
m = Chem.MolFromSmiles(smiles[1], sanitize=False)
Chem.SanitizeMol(m,sanitizeOps=Chem.SanitizeFlags.SANITIZE_ALL^Chem.SanitizeFlags.SANITIZE_KEKULIZE)
m.UpdatePropertyCache()
fp = AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=768, bitInfo=bi)
m

In [ ]:
mfp = np.array(fp)
sorted_bits = []
for i in sortidx[::-1]:
  if mfp[i] == 1: sorted_bits.append(i)
print(sorted_bits)

In [ ]:
Draw.DrawMorganBit(m, sorted_bits[1], bi)

In [ ]:
Draw.DrawMorganBit(m, sorted_bits[2], bi)

In [ ]:
Draw.DrawMorganBit(m, sorted_bits[4], bi)

In [ ]:
Draw.DrawMorganBit(m, sorted_bits[5], bi)

In [ ]:
Draw.DrawMorganBit(m, sorted_bits[7], bi)